In [1]:
import pandas as pd
import numpy as np

import featuretools as ft

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor, XGBRFRegressor
from keras import models
from keras import layers
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [34]:
train = pd.read_csv('/Users/hduser/Documents/AnalyticsVidhya/Hackathons/HackerEarth/1/Dataset/Train.csv')
test = pd.read_csv('/Users/hduser/Documents/AnalyticsVidhya/Hackathons/HackerEarth/1/Dataset/Test.csv')
y = train.Attrition_rate
ID = test.Employee_ID
train.drop(['Attrition_rate'], axis=1, inplace=True)
data = pd.concat([train, test])
data.drop(['Employee_ID'], axis=1, inplace=True)

# Filling Missing Data

In [35]:
data.isnull().sum()

Gender                         0
Age                          573
Education_Level                0
Relationship_Status            0
Hometown                       0
Unit                           0
Decision_skill_possess         0
Time_of_service              196
Time_since_promotion           0
growth_rate                    0
Travel_Rate                    0
Post_Level                     0
Pay_Scale                     12
Compensation_and_Benefits      0
Work_Life_balance             16
VAR1                           0
VAR2                         794
VAR3                           0
VAR4                         954
VAR5                           0
VAR6                           0
VAR7                           0
dtype: int64

In [36]:
def age_fill(data):
    data.Age.fillna('NaN', inplace=True)
    age_sam = data.Age == 'NaN'
    
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Time_of_service >= 0) & (data.Time_of_service < 5) ] = 30
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Time_of_service >= 5) & (data.Time_of_service < 20) ] = 40
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Time_of_service >= 20) & (data.Time_of_service < 30) ] = 50
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Time_of_service >= 30) & (data.Time_of_service < 44) ] = 60
    
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Time_of_service >= 0) & (data.Time_of_service < 5) ] = 45
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Time_of_service >= 5) & (data.Time_of_service < 20) ] = 55
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Time_of_service >= 20) & (data.Time_of_service < 30) ] = 60
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Time_of_service >= 30) & (data.Time_of_service < 44) ] = 65
    
age_fill(data)

In [37]:
def age_fill_tos(data):
    age_sam = data.Age == 'NaN'
    
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Pay_Scale >= 1) & (data.Pay_Scale < 4) ] = 30
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Pay_Scale >= 4) & (data.Pay_Scale < 7) ] = 37
    data.Age[(age_sam) & (data.Education_Level >= 1) & (data.Education_Level < 3) & 
             (data.Pay_Scale >= 7) & (data.Pay_Scale < 11) ] = 42
    
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Pay_Scale >= 1) & (data.Pay_Scale < 4) ] = 35
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Pay_Scale >= 4) & (data.Pay_Scale < 7) ] = 45
    data.Age[(age_sam) & (data.Education_Level >= 3) & (data.Education_Level < 6) & 
             (data.Pay_Scale >= 7) & (data.Pay_Scale < 11) ] = 55

age_fill_tos(data)

data.Age = pd.to_numeric(data.Age, errors='coerce')

In [38]:
def timeofservice(data):
    data.Time_of_service.fillna('NaN', inplace=True)
    t_sam = data.Time_of_service == 'NaN'
    
    data.Time_of_service[(t_sam) & (data.Age >= 19) & (data.Age < 30) & 
                        (data.Education_Level >= 1) & (data.Education_Level < 3)] = 5
    data.Time_of_service[(t_sam) & (data.Age >= 19) & (data.Age < 30) & 
                        (data.Education_Level >= 3) & (data.Education_Level < 5)] = 3
    data.Time_of_service[(t_sam) & (data.Age >= 19) & (data.Age < 30) & 
                        (data.Education_Level >= 5) & (data.Education_Level < 6)] = 2
    
    data.Time_of_service[(t_sam) & (data.Age >= 30) & (data.Age < 40) & 
                        (data.Education_Level >= 1) & (data.Education_Level < 3)] = 12
    data.Time_of_service[(t_sam) & (data.Age >= 30) & (data.Age < 40) & 
                        (data.Education_Level >= 3) & (data.Education_Level < 5)] = 10
    data.Time_of_service[(t_sam) & (data.Age >= 30) & (data.Age < 40) & 
                        (data.Education_Level >= 5) & (data.Education_Level < 6)] = 9
    
    data.Time_of_service[(t_sam) & (data.Age >= 40) & (data.Age < 55) & 
                        (data.Education_Level >= 1) & (data.Education_Level < 3)] = 24
    data.Time_of_service[(t_sam) & (data.Age >= 40) & (data.Age < 55) & 
                        (data.Education_Level >= 3) & (data.Education_Level < 5)] = 20
    data.Time_of_service[(t_sam) & (data.Age >= 40) & (data.Age < 55) & 
                        (data.Education_Level >= 5) & (data.Education_Level < 6)] = 18
    
    data.Time_of_service[(t_sam) & (data.Age >= 55) & (data.Age < 66) & 
                        (data.Education_Level >= 1) & (data.Education_Level < 3)] = 38
    data.Time_of_service[(t_sam) & (data.Age >= 55) & (data.Age < 66) & 
                        (data.Education_Level >= 3) & (data.Education_Level < 5)] = 35
    data.Time_of_service[(t_sam) & (data.Age >= 55) & (data.Age < 66) & 
                        (data.Education_Level >= 5) & (data.Education_Level < 6)] = 32

timeofservice(data)

data.Time_of_service = pd.to_numeric(data.Time_of_service, errors='coerce')

In [39]:
data.Pay_Scale.fillna(8, inplace=True)
data.Work_Life_balance.fillna(1, inplace=True)
data.VAR2.fillna(0.7516, inplace=True)
data.VAR4.fillna(2, inplace=True)

In [40]:
data

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,...,7.0,type2,3.0,4,0.7516,1.8688,2.0,4,5,3
1,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,...,6.0,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3
2,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,...,8.0,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3
3,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,...,8.0,type2,1.0,3,-1.8176,-0.4537,2.0,3,7,3
4,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,62,...,2.0,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2
5,M,54.0,3,Married,Lebanon,Purchasing,Conceptual,19.0,1,21,...,9.0,type2,1.0,3,-1.8176,1.8688,2.0,2,8,3
6,F,21.0,4,Married,Springfield,Purchasing,Directive,2.0,1,50,...,6.0,type2,2.0,3,-0.9612,0.7075,2.0,3,7,3
7,F,65.0,3,Married,Springfield,Sales,Directive,34.0,4,23,...,6.0,type3,2.0,3,-0.1048,-0.4537,2.0,3,9,3
8,M,40.0,4,Single,Springfield,Production,Analytical,13.0,1,49,...,9.0,type0,4.0,1,0.7516,1.8688,2.0,5,6,3
9,M,45.0,3,Married,Lebanon,IT,Directive,21.0,4,34,...,5.0,type3,4.0,3,0.7516,-0.4537,2.0,4,8,3


# Data Preprocessing

In [8]:
def typ(s):
    if s == 'type0':
        return 1
    elif s == 'type1':
        return 2
    elif s == 'type2':
        return 3
    elif s == 'type3':
        return 4
    elif s == 'type4':
        return 5
    elif s == 'type5':
        return 6

data.Compensation_and_Benefits = data.Compensation_and_Benefits.apply(typ)

## Feature Generation

In [9]:
bins=[18,30,40,50,60,76]
group=[1,2,3,4,5]
data['age_category']=pd.cut(data['Age'], bins, labels=group)

In [10]:
data['value'] = ((data.Education_Level * data.Time_of_service) + data.Pay_Scale) / data.Age
data['some_rate'] = (data.growth_rate * data.Time_of_service) / 100
data['Total_earning'] = data.Pay_Scale * data.Time_of_service 
data['balance'] = (data.Compensation_and_Benefits * data.Pay_Scale) / (data.Post_Level * (data.Travel_Rate+1))
#data['Post_Pay'] = data.Post_Level * data.Pay_Scale
#data['Without_promotion'] = data.Time_of_service - data.Time_since_promotion


## VAR2 and VAR3

In [2222]:
VAR2 = data.VAR2.unique()
VAR2_sorted = np.sort(VAR2)
                           
np.diff(VAR2_sorted)

array([0.8564, 0.8564, 0.8564, 0.8565])

In [2223]:
np.diff(VAR2_sorted/0.8564)

array([1.        , 1.        , 1.        , 1.00011677])

In [2224]:
data.VAR2= (data.VAR2/0.8564 - .122373).round() + 3

In [2225]:
data.VAR2.value_counts()

4.0    4376
3.0    2288
2.0    1469
1.0    1173
5.0     694
Name: VAR2, dtype: int64

In [2226]:
VAR3 = data.VAR3.unique()
VAR3_sorted = np.sort(VAR3)
                           
np.diff(VAR3_sorted)

array([1.1612, 1.1613, 1.1612, 1.1613])

In [2227]:
data.VAR3 = (data.VAR3/1.1612 - .609370).round() + 4

In [2228]:
data.VAR3.value_counts()

3.0    4755
4.0    3362
5.0     888
2.0     709
1.0     286
Name: VAR3, dtype: int64

## Frequency Encoding

In [11]:
def frequency_encoding(data, col):
    encoding = data.groupby(col).size()
    encoding = encoding / len(data)
    final = 'enc_' + col
    data[final] = data[col].map(encoding)
    
    data.drop(col, axis=1, inplace=True)
    
    return encoding

## Mean Encoding

In [12]:
def mean_encoding(train, col, target, test):
    mean = train.groupby(col)[target].mean()
    train[col] = train[col].map(mean)
    test[col] = test[col].map(mean)
    return mean

#mean_encoding(train_dt, 'Gender', 'Attrition_rate', test_dt)

In [13]:
'''train_dt = data[:7000]
test_dt = data[7000:]
train_dt['Attrition_rate'] = y'''

"train_dt = data[:7000]\ntest_dt = data[7000:]\ntrain_dt['Attrition_rate'] = y"

### Gender

In [14]:
frequency_encoding(data, 'Gender')
#mean_encoding(train_dt, 'Gender', 'Attrition_rate', test_dt)

Gender
F    0.5842
M    0.4158
dtype: float64

### Relationship_Status

In [15]:
frequency_encoding(data, 'Relationship_Status')
#mean_encoding(train_dt, 'Relationship_Status', 'Attrition_rate', test_dt)

Relationship_Status
Married    0.6461
Single     0.3539
dtype: float64

### Unit

In [16]:
frequency_encoding(data, 'Unit')
#mean_encoding(train_dt, 'Unit', 'Attrition_rate', test_dt)

Unit
Accounting and Finance       0.0679
Human Resource Management    0.0513
IT                           0.1963
Logistics                    0.1648
Marketing                    0.0319
Operarions                   0.1008
Production                   0.0292
Purchasing                   0.0714
Quality                      0.0290
R&D                          0.0991
Sales                        0.1389
Security                     0.0194
dtype: float64

### Hometown

In [17]:
frequency_encoding(data, 'Hometown')
#mean_encoding(train_dt, 'Hometown', 'Attrition_rate', test_dt)

Hometown
Clinton        0.0812
Franklin       0.2181
Lebanon        0.2953
Springfield    0.2484
Washington     0.1570
dtype: float64

### Decision_skill_possess

In [18]:
frequency_encoding(data, 'Decision_skill_possess')
#mean_encoding(train_dt, 'Decision_skill_possess', 'Attrition_rate', test_dt)

Decision_skill_possess
Analytical    0.2501
Behavioral    0.2535
Conceptual    0.2476
Directive     0.2488
dtype: float64

### Education Level

In [19]:
frequency_encoding(data, 'Education_Level')
#mean_encoding(train_dt, 'Education_Level', 'Attrition_rate', test_dt)

Education_Level
1    0.0979
2    0.0982
3    0.4063
4    0.3018
5    0.0958
dtype: float64

### Travel_Rate

In [20]:
frequency_encoding(data, 'Travel_Rate')
#mean_encoding(train_dt, 'Travel_Rate', 'Attrition_rate', test_dt)

Travel_Rate
0    0.3147
1    0.5499
2    0.1354
dtype: float64

### Compensation_and_Benefits

In [21]:
frequency_encoding(data, 'Compensation_and_Benefits')
#mean_encoding(train_dt, 'Compensation_and_Benefits', 'Attrition_rate', test_dt)

Compensation_and_Benefits
1    0.0280
2    0.0198
3    0.5686
4    0.3346
5    0.0490
dtype: float64

### Work_Life_balance

In [22]:
frequency_encoding(data, 'Work_Life_balance')
#mean_encoding(train_dt, 'Work_Life_balance', 'Attrition_rate', test_dt)

Work_Life_balance
1.0    0.2967
2.0    0.2287
3.0    0.2858
4.0    0.1719
5.0    0.0169
dtype: float64

## Age Binning

In [23]:
frequency_encoding(data, 'age_category')
#mean_encoding(train_dt, 'age_category', 'Attrition_rate', test_dt)

age_category
1    0.3152
2    0.2006
3    0.1894
4    0.2273
5    0.0675
dtype: float64

### VAR 1 through 7

In [24]:
frequency_encoding(data, 'VAR1')
#frequency_encoding(data, 'VAR2')
#frequency_encoding(data, 'VAR3')
frequency_encoding(data, 'VAR4')
frequency_encoding(data, 'VAR5')
frequency_encoding(data, 'VAR6')
frequency_encoding(data, 'VAR7')

'''mean_encoding(train_dt, 'VAR1', 'Attrition_rate', test_dt)
mean_encoding(train_dt, 'VAR2', 'Attrition_rate', test_dt)
mean_encoding(train_dt, 'VAR3', 'Attrition_rate', test_dt)
mean_encoding(train_dt, 'VAR4', 'Attrition_rate', test_dt)
mean_encoding(train_dt, 'VAR5', 'Attrition_rate', test_dt)
mean_encoding(train_dt, 'VAR6', 'Attrition_rate', test_dt)
mean_encoding(train_dt, 'VAR7', 'Attrition_rate', test_dt)'''

"mean_encoding(train_dt, 'VAR1', 'Attrition_rate', test_dt)\nmean_encoding(train_dt, 'VAR2', 'Attrition_rate', test_dt)\nmean_encoding(train_dt, 'VAR3', 'Attrition_rate', test_dt)\nmean_encoding(train_dt, 'VAR4', 'Attrition_rate', test_dt)\nmean_encoding(train_dt, 'VAR5', 'Attrition_rate', test_dt)\nmean_encoding(train_dt, 'VAR6', 'Attrition_rate', test_dt)\nmean_encoding(train_dt, 'VAR7', 'Attrition_rate', test_dt)"

In [25]:
#train_dt.drop(['Attrition_rate'], axis=1, inplace=True)

In [26]:
#data = pd.concat([train_dt, test_dt])

# Data Transformation

In [27]:
data.Time_of_service = np.log(data.Time_of_service+1)
data.value = data.value.apply(np.log)
data.some_rate = np.log(data.some_rate+1)
data.Total_earning = data.Total_earning.apply(np.sqrt)

# Data Scaling

In [28]:
'''es = ft.EntitySet(id = 'attrition')
features = data.copy()
es.entity_from_dataframe(entity_id = 'data_1', dataframe = features, 
                         make_index = True, index = 'index')

feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data_1', max_depth=1,
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])

feature_matrix.head()'''

"es = ft.EntitySet(id = 'attrition')\nfeatures = data.copy()\nes.entity_from_dataframe(entity_id = 'data_1', dataframe = features, \n                         make_index = True, index = 'index')\n\nfeature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data_1', max_depth=1,\n                                      trans_primitives = ['add_numeric', 'multiply_numeric'])\n\nfeature_matrix.head()"

In [29]:
'''idx = data.columns

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
scaled_data = pd.DataFrame(scaled_data, columns=idx)

copy_scaled_data = scaled_data.copy()
train_final = copy_scaled_data[0:7000]
test_final = copy_scaled_data[7000:]
'''
idx = data.columns
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
scaled_data = pd.DataFrame(scaled_data, columns=idx)
copy_scaled_data = scaled_data.copy()
train_final = copy_scaled_data[0:7000]
test_final = copy_scaled_data[7000:]

'''idx = feature_matrix.columns
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(feature_matrix)
scaled_data = pd.DataFrame(scaled_data, columns=idx)
copy_scaled_data = scaled_data.copy()
train_final = copy_scaled_data[0:7000]
test_final = copy_scaled_data[7000:]'''

'idx = feature_matrix.columns\nscaler = MinMaxScaler()\nscaled_data = scaler.fit_transform(feature_matrix)\nscaled_data = pd.DataFrame(scaled_data, columns=idx)\ncopy_scaled_data = scaled_data.copy()\ntrain_final = copy_scaled_data[0:7000]\ntest_final = copy_scaled_data[7000:]'

In [30]:
len(train) == len(train_final) == len(y)

True

# Neural Network Model

In [1079]:
network = models.Sequential()
network.add(layers.Dense(70, activation='relu', input_dim=27))
network.add(layers.Dense(70, activation='relu'))
network.add(layers.Dense(70, activation='relu'))
network.add(layers.Dense(70, activation='relu'))
network.add(layers.Dense(1)) 

In [1080]:
network.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['accuracy'])

In [1081]:
network.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_134 (Dense)            (None, 70)                1960      
_________________________________________________________________
dense_135 (Dense)            (None, 70)                4970      
_________________________________________________________________
dense_136 (Dense)            (None, 70)                4970      
_________________________________________________________________
dense_137 (Dense)            (None, 70)                4970      
_________________________________________________________________
dense_138 (Dense)            (None, 1)                 71        
Total params: 16,941
Trainable params: 16,941
Non-trainable params: 0
_________________________________________________________________


In [1082]:
network.fit(train_final, y, epochs=200, verbose=1)

Epoch 1/200
7000/7000 [==============================] - 0s 60us/step - loss: 0.0419 - accuracy: 5.7143e-04
Epoch 2/200
7000/7000 [==============================] - 0s 33us/step - loss: 0.0345 - accuracy: 5.7143e-04
Epoch 3/200
7000/7000 [==============================] - 0s 32us/step - loss: 0.0331 - accuracy: 5.7143e-04
Epoch 4/200
7000/7000 [==============================] - 0s 31us/step - loss: 0.0319 - accuracy: 5.7143e-04
Epoch 5/200
7000/7000 [==============================] - 0s 31us/step - loss: 0.0311 - accuracy: 5.7143e-04
Epoch 6/200
7000/7000 [==============================] - 0s 31us/step - loss: 0.0297 - accuracy: 5.7143e-04
Epoch 7/200
7000/7000 [==============================] - 0s 32us/step - loss: 0.0283 - accuracy: 5.7143e-04
Epoch 8/200
7000/7000 [==============================] - 0s 33us/step - loss: 0.0268 - accuracy: 5.7143e-04
Epoch 9/200
7000/7000 [==============================] - 0s 32us/step - loss: 0.0250 - accuracy: 5.7143e-04
Epoch 10/200
7000/7000 [====

7000/7000 [==============================] - 0s 30us/step - loss: 0.0020 - accuracy: 5.7143e-04
Epoch 151/200
7000/7000 [==============================] - 0s 31us/step - loss: 0.0022 - accuracy: 5.7143e-04
Epoch 152/200
7000/7000 [==============================] - 0s 41us/step - loss: 0.0018 - accuracy: 5.7143e-04
Epoch 153/200
7000/7000 [==============================] - 0s 43us/step - loss: 0.0016 - accuracy: 5.7143e-04
Epoch 154/200
7000/7000 [==============================] - 0s 47us/step - loss: 0.0012 - accuracy: 5.7143e-04
Epoch 155/200
7000/7000 [==============================] - 0s 47us/step - loss: 0.0012 - accuracy: 5.7143e-04
Epoch 156/200
7000/7000 [==============================] - 0s 32us/step - loss: 0.0013 - accuracy: 5.7143e-04
Epoch 157/200
7000/7000 [==============================] - 0s 31us/step - loss: 0.0014 - accuracy: 5.7143e-04
Epoch 158/200
7000/7000 [==============================] - 0s 31us/step - loss: 0.0015 - accuracy: 5.7143e-04
Epoch 159/200
7000/7000 

# XGB

In [1153]:
clf = XGBRegressor(random_state=96, colsample_bytree=0.7, max_depth=6, n_jobs=-1, gamma=0.1)

In [1154]:
clf.fit(train_final, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=96, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [31]:
clf2 = XGBRFRegressor(random_state=96, colsample_bytree=0.7, max_depth=6, n_jobs=-1, gamma=0.1, 
                      reg_alpha=.5)

In [1982]:
we = []
for i in y:
    i += 0.0001
    num = 1/(i**2)
    we.append(num)

denum = np.sum(we)
w = we/denum

# only for MSPE, MAE, MSE

In [32]:
clf2.fit(train_final, y)

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=0.8, colsample_bytree=0.7, gamma=0.1, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=1, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=-1, num_parallel_tree=100,
               objective='reg:squarederror', random_state=96, reg_alpha=0.5,
               reg_lambda=1e-05, scale_pos_weight=1, subsample=0.8,
               tree_method='exact', validate_parameters=1, verbosity=None)

# AdaBoost

In [1710]:
ada = AdaBoostRegressor(random_state=96, base_estimator=RandomForestRegressor(random_state=101, n_jobs=-1,
                                                                             max_depth=10),
                         n_estimators=100, learning_rate=0.01)

In [ ]:
ada.fit(train_final, y)

# LGBM

In [1356]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmsle',
    'max_depth': 6, 
    'learning_rate': 0.1,
    'verbose': 0}
n_estimators = 100

In [1361]:
'''x_train, x_valid, y_train, y_valid = train_test_split(train_final, y, test_size=0.10, random_state=5)
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)
watchlist = [d_valid]'''
d_train = lgb.Dataset(train_final, label=y)

In [1362]:
model = lgb.train(params, d_train, n_estimators, verbose_eval=1)

# Output

In [33]:
final = clf2.predict(test_final)
dv = pd.DataFrame(ID)
dv['Attrition_rate'] = final
dv.to_csv('sample_submission.csv', index=False)